## Sentinel-2 Mineral RGB Composite

### <a name="service">Service definition

In [1]:
service = dict([('title', 'Sentinel-2 Mineral RGB Composite'),
                ('abstract', 'Sentinel-2 Mineral RGB Composite'),
                ('id', 'ewf-s2-mineral-composite')])

### Service parameters

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the Sentinel-1 stack of products' identifiers

In [2]:
input_identifiers = ['A8204ADAC1D07DE1AAFCC0C87CFDB6CB1DDAD310',
                     '57D0F4788C1C7DB2898D9F1C256F2D391B184453'] 


In [3]:
aoi = dict([('id', 'aoi'),
            ('title', 'Area of interest (bbox)'),
            ('abstract', 'Area of interest defined as a bounding box'),
            ('value', '-70.5659,-13.0922,-69.1411,-12.4567')])

**Input references**

This is the Sentinel-1 stack catalogue references

In [4]:
input_references = ['https://catalog.terradue.com/sen2cor/search?format=atom&uid=A8204ADAC1D07DE1AAFCC0C87CFDB6CB1DDAD310',
                     'https://catalog.terradue.com/sen2cor/search?format=atom&uid=57D0F4788C1C7DB2898D9F1C256F2D391B184453'] 

**Data path**

This path defines where the data is staged-in. 

In [5]:
data_path = '/workspace/data'

In [6]:
username = dict([('id', '_T2Username'),
              ('title', 'Ellip username'),
              ('abstract', 'Ellip username'),
              ('value', '')])

In [7]:
api_key = dict([('id', '_T2ApiKey'),
              ('title', 'Ellip API key for data pipeline'),
              ('abstract', 'Ellip API key for data pipeline'),
              ('value', '')])

### <a name="workflow">Workflow

In [8]:
import cioppy
import geopandas as gpd
from shapely.wkt import loads
import pandas as pd
import os
import osr
import sys
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())

from helpers import *

sys.path.append('/opt/OTB/lib/python')
sys.path.append('/opt/OTB/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB/lib/otb/applications'

import otbApplication
import requests
%load_ext autoreload
%autoreload 2

In [9]:
ciop = cioppy.Cioppy()

In [10]:
creds = '{0}:{1}'.format(username['value'], api_key['value'])

In [11]:
for index, reference in enumerate(input_references):
    print reference

https://catalog.terradue.com/sen2cor/search?format=atom&uid=A8204ADAC1D07DE1AAFCC0C87CFDB6CB1DDAD310


In [12]:
                                        
                                           
temp_searches = []

for index, reference in enumerate(input_references):

    temporary_item =ciop.search(end_point=reference,
                                      params=[],
                                       output_fields='self,track,enclosure,identifier,wkt,startdate,enddate,platform,cc', 
                                       model='EOP',creds=creds)

    
        
    search_temp = gpd.GeoDataFrame(temporary_item)
    if 'https://catalog.terradue.com/sentinel2/' in temporary_item[0]['self'] :
        original_identifier= temporary_item[0]['identifier']   
        
    else: 
        original_identifier = get_original_identifier(temporary_item[0]['self'],username['value'], api_key['value'])
    search_temp['local_path'] = os.path.join(data_path, original_identifier)
    search_temp['original_identifier'] = original_identifier
    temp_searches.append(search_temp)
    
search = gpd.GeoDataFrame(pd.concat(temp_searches, ignore_index=True)) 

search['geometry'] = search['wkt'].apply(loads)
search['cc'] = pd.to_numeric(search['cc'])
search['startdate'] = pd.to_datetime(search['startdate'])
search['enddate'] = pd.to_datetime(search['enddate'])                                           
                                           
                                           

    
    

In [13]:
search

,cc,enclosure,enddate,identifier,platform,self,startdate,track,wkt,local_path,original_identifier,geometry
0,0.099581,https://store.terradue.com/sen2cor/_results/wo...,2018-08-13 14:57:19.024,A8204ADAC1D07DE1AAFCC0C87CFDB6CB1DDAD310,S2B,https://catalog.terradue.com/sen2cor/search?fo...,2018-08-13 14:57:19.024,039,"POLYGON((-69.42706 -11.7587371884498,-69.43216...",/workspace/data/S2B_MSIL2A_20180813T145719_N02...,S2B_MSIL2A_20180813T145719_N0206_R039_T19LDG_2...,"POLYGON ((-69.42706 -11.7587371884498, -69.432..."


In [ ]:
search = search.merge(search.apply(lambda row: analyse(row ), axis=1),
                                    left_index=True,
                                  right_index=True)

In [ ]:
search

In [ ]:


aoi_wkt = box(*[float(i) for i in aoi['value'].split(',')]).wkt

min_lon, min_lat,  max_lon, max_lat = [float(i) for i in aoi['value'].split(',')]


In [ ]:
bands = ['B02', 'B04', 'B11', 'B12', 'SCL']

In [ ]:
for index, band in enumerate(bands):
    
    vrt_bands = []
    
    for j, row in search.iterrows():
        
        vrt_bands.append(get_band_path(row, band))
    
    vrt = '{0}.vrt'.format(band)
    ds = gdal.BuildVRT(vrt,
                       vrt_bands,
                       #srcNodata=0,
                       xRes=10, 
                       yRes=10)

    ds.FlushCache()
    
    
    gdal.Translate(band + '.tif', 
                   ds, 
                   outputType=gdal.GDT_Float32, 
                   projWin=[min_lon, max_lat, max_lon, min_lat],
                   projWinSRS='EPSG:4326')

In [ ]:
r_channel = 'im4b1 !=0 ? im3b1 / im4b1 : 0'
g_channel = 'im1b1 !=0 ? im2b1 / im1b1 : 0'
b_channel = 'im3b1 !=0 ? im2b1 / im3b1 : 0'
alpha_channel = '{0} == 0 || {0} == 1 || {0} == 3 || {0} == 8 || {0} == 9 ? 0 : 1'.format('im5b1') 

In [ ]:
band_expressions = [r_channel, 
                    g_channel, 
                    b_channel,
                    alpha_channel]

In [ ]:
BandMathX = otbApplication.Registry.CreateApplication("BandMathX")

BandMathX.SetParameterStringList('il', [b + '.tif' for b in bands])

BandMathX.SetParameterString('out', 'temp.tif')

BandMathX.SetParameterString('exp', ';'.join(band_expressions))

BandMathX.ExecuteAndWriteOutput()

In [ ]:
output_startdate = min(search['startdate'])
output_stopdate = max(search['enddate'])

In [ ]:
date_format = '%Y%m%dT%H%m%S'

output_name = 'MINERAL-COMPOSITE-{0}-{1}'.format(output_startdate.strftime(date_format), 
                                                 output_stopdate.strftime(date_format))

In [ ]:
Convert = otbApplication.Registry.CreateApplication('Convert')


Convert.SetParameterString('in', 'temp.tif')

Convert.SetParameterString('out', 'temp_{}.tif'.format(output_name))

Convert.SetParameterString('type', 'linear')

Convert.SetParameterString('channels', 'all')

Convert.ExecuteAndWriteOutput()

In [ ]:
cog('temp_{}.tif'.format(output_name),
    '{}.tif'.format(output_name))

### Clean-up

In [ ]:
for band in bands:
    for extension in ['.tif', '.vrt']:
        os.remove(band + extension)
        
os.remove('temp.tif')

 ### Create RGB (4bands) with Mask of changes (0-255 values)


In [ ]:
from helpers import *

In [ ]:
out_rgb = '{}-rgb'.format(output_name)

create_rgb('{}.tif'.format(output_name), '{}.tif'.format(out_rgb))

### Metadata about results


In [ ]:
for properties_file in ['result', 'stage-in', output_name, out_rgb]:

    date_format = '%Y-%m-%dT%H:%m:%SZ'
    
    if properties_file == 'result':
        
        title = 'Reproducibility notebook used for generating {0}'.format(output_name)
   
    elif properties_file == 'stage-in':

        title = 'Reproducibility stage-in notebook for Sentinel-2 data for generating {0}'.format(output_name)
    
    
        
    elif properties_file == out_rgb:
        
        title = 'RGB composite for {0} to {1}'.format(output_startdate.strftime(date_format),
                                                                  output_stopdate.strftime(date_format))
    
    else: 
      
        title = 'Mineral alteration index from {0} to {1}'.format(output_startdate.strftime(date_format),
                                                                  output_stopdate.strftime(date_format))
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title={0}\n'.format(title))
        file.write('date={0}/{1}\n'.format(output_startdate.strftime(date_format),
                                           output_stopdate.strftime(date_format)))
        file.write('geometry={0}'.format(get_image_wkt(output_name + '.tif')))


### License




This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.